In [1]:
import json
import random
from transformers import AutoTokenizer
import numpy as np
from allennlp.common.util import import_module_and_submodules as import_submodules
from allennlp.models.archival import load_archive
from allennlp.predictors import Predictor
from scipy.spatial import distance

import sys
import os
sys.path.append(os.path.abspath('..'))

import_submodules("allennlp_lib")

DATASET="ecthr"
MODEL_NAME="allenai/longformer-base-4096"
model_path=f"../experiments/models/{DATASET}/{MODEL_NAME}"

archive = load_archive(model_path + '/model.tar.gz')
print(archive.config)
archive.config['dataset_reader']['type'] = 'ecthr'
archive.config['model']['output_hidden_states'] = True
model = archive.model
model._output_hidden_states = True
predictor = Predictor.from_archive(archive, 'ecthr')

tok = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

with open(model_path + "/label2index.json", "r") as f:
    label2index = json.load(f)
    index2label = {label2index[k]: k for k in label2index}
label2index


/home/irs38/.conda/envs/contrastive/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 694/694 [00:00<00:00, 2.87MB/s]
Downloading: 100%|██████████| 597M/597M [00:06<00:00, 96.9MB/s] 
Downloading: 100%|██████████| 899k/899k [00:00<00:00, 12.1MB/s]
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 5.65MB/s]
Downloading: 100%|██████████| 1.36M/1.36M [00:00<00:00, 17.1MB/s]


Params({'dataset_reader': {'token_indexers': {'tokens': {'max_length': 4096, 'model_name': 'allenai/longformer-base-4096', 'type': 'pretrained_transformer'}}, 'tokenizer': {'add_special_tokens': False, 'model_name': 'allenai/longformer-base-4096', 'type': 'pretrained_transformer'}, 'type': 'ecthr'}, 'trainer': {'cuda_device': 0, 'learning_rate_scheduler': {'cut_frac': 0.06, 'type': 'slanted_triangular'}, 'num_epochs': 2, 'optimizer': {'lr': 2e-06, 'type': 'huggingface_adamw', 'weight_decay': 0.1}, 'patience': 5, 'use_amp': True, 'validation_metric': '+accuracy'}, 'evaluate_on_test': True, 'train_data_path': 'data/ecthr/Chalkidis/simple_train.jsonl', 'validation_data_path': 'data/ecthr/Chalkidis/simple_val.jsonl', 'test_data_path': 'data/ecthr/Chalkidis/simple_test.jsonl', 'data_loader': {'batch_sampler': {'batch_size': 1, 'type': 'bucket'}}, 'model': {'dropout': 0.1, 'feedforward': {'activations': 'tanh', 'hidden_dims': 768, 'input_dim': 768, 'num_layers': 1}, 'namespace': 'tags', 'seq

{'not_claimed': 0, 'claimed_and_violated': 1, 'claimed_not_violated': 2}

In [2]:
def all_masks(tokenized_text):
    # https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset
    # WITHOUT empty and full sets!
    s = list(range(len(tokenized_text)))
    x = len(s)
    masks = [1 << i for i in range(x)]
    #     for i in range(1 << x):  # empty and full sets included here
    for i in range(1, 1 << x - 1):
        yield [ss for mask, ss in zip(masks, s) if i & mask]
        
def all_consecutive_masks(tokenized_text, max_length = -1):
    # WITHOUT empty and full sets!
    s = list(range(len(tokenized_text)))
    x = len(s)
    for i in range(x):
        for j in range(i+1, x):
            mask = s[:i] + s[j:]
            if max_length > 0:
                if j - i >= max_length:
                    yield mask
            else:
                yield mask
                
def all_consecutive_masks2(tokenized_text, max_length = -1):
    # WITHOUT empty and full sets!
    s = list(range(len(tokenized_text)))
    x = len(s)
    for i in range(x+1):
        for j in range(i+1, x+1):
            mask = s[i:j]
            if max_length > 0:
                if j - i <= max_length:
                    yield mask
            else:
                yield mask


In [4]:

ex = {"facts": "5.  The applicant was born in 1983 and is detained in Sztum. 6.  At the time of the events in question, the applicant was serving a prison sentence in the Barczewo prison. 7.  On 8 January 2011 the applicant\u2019s grandmother died. On 10 January 2011 the applicant lodged a request with the Director of Prison and the Penitentiary judge for leave to attend her funeral which was to take place on 12 January 2011. Together with his application he submitted a statement from his sister E.K. who confirmed that she would personally collect the applicant from prison and bring him back after the funeral. 8.  On 11 January 2011 the Penitentiary judge of the Olsztyn Regional Court (S\u0119dzia Penitencjarny S\u0105du Okr\u0119gowego w Olsztynie) allowed the applicant to attend the funeral under prison officers\u2019 escort. The reasoning of the decision read as follows:\n\u201cIn view of [the applicant\u2019s] multiple convictions and his long term of imprisonment there is no guarantee that he will return to prison\u201d 9.  The applicant refused to attend the funeral, since he believed his appearance under escort of uniformed officers would create a disturbance during the ceremony. 10.  On the same day the applicant lodged an appeal with the Olsztyn Regional Court (S\u0105d Okr\u0119gowy) complaining that the compassionate leave was granted under escort and also that he was only allowed to participate in the funeral (not the preceding church service). 11.  On 3 February 2011 the Olsztyn Regional Court upheld the Penitentiary judge\u2019s decision and dismissed the appeal. The court stressed that the applicant had been allowed to participate in the funeral under prison officers\u2019 escort. It further noted that the applicant was a habitual offender sentenced to a long term of imprisonment therefore there was no positive criminological prognosis and no guarantee that he would have returned to prison after the ceremony.", "claims": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], "outcomes": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "case_no": "20488/11"}

claims = ex["claims"]
outcomes = ex["outcomes"]
foils = [0 if c == 0 else 1 if c == 1 and o == 1 else 2 for c, o in zip(claims, outcomes)]

foil = ex['gold_label']

out = predictor.predict_json(ex)
encoded_orig = out['encoded_representations']

fact = out['label']
print('Predicted: ', fact)

# assert fact != foil, "Fact should be different from the foil (if not, pick a different foil)"

ex['sentence1'] = ex['sentence1'].split()
ex['sentence2'] = ex['sentence2'].split()

tok.convert_tokens_to_string(out['tokens'])

masks1 = [[]]  # change this if you also want to mask out parts of the premise.
masks2 = list(all_consecutive_masks2(ex['sentence2'], max_length=1))
encoded = []
mask_mapping = []
preds = np.zeros(shape=(len(masks1), len(masks2)))

for m1_i, m1 in enumerate(masks1):
    masked1 = list(ex['sentence1'])
    for i in m1:
        masked1[i] = '<mask>'
    masked1 = ' '.join(masked1)
        
    for m2_i, m2 in enumerate(masks2):
        masked2 = list(ex['sentence2'])
        for i in m2:
            masked2[i] = '<mask>'
        masked2 = ' '.join(masked2)
            
        masked_ex = {
            "sentence1": masked1,
            "sentence2": masked2
        }
        
        masked_out = predictor.predict_json(masked_ex)
#         if masked_out['label'] != foil:
#             continue
        
        print(m1_i, m2_i)
        print(f"{masked1}\n{masked2}")
        print(masked_out['label'])
        encoded.append(masked_out['encoded_representations'])
        mask_mapping.append((m1_i, m2_i))
        
        print("====")
        
encoded = np.array(encoded)
        

In [ ]:
foil = 'neutral'

fact_idx = label2index[fact]
foil_idx = label2index[foil]
print('fact:', index2label[fact_idx])
print('foil:', index2label[foil_idx])
num_classifiers = 100

classifier_w = np.load(f"{model_path}/w.npy")
classifier_b = np.load(f"{model_path}/b.npy")

u = classifier_w[fact_idx] - classifier_w[foil_idx]
contrastive_projection = np.outer(u, u) / np.dot(u, u)

print(contrastive_projection.shape)


In [ ]:

# from scipy.stats import entropy
from scipy.special import softmax

z_all = encoded_orig 
z_h = encoded 
z_all_row = encoded_orig @ contrastive_projection
z_h_row = encoded @ contrastive_projection

prediction_probabilities = softmax(z_all_row @ classifier_w.T + classifier_b)
prediction_probabilities = np.tile(prediction_probabilities, (z_h_row.shape[0], 1))

prediction_probabilities_del = softmax(z_h_row @ classifier_w.T + classifier_b, axis=1)

p = prediction_probabilities[:, [fact_idx, foil_idx]]
q = prediction_probabilities_del[:, [fact_idx, foil_idx]]

p = p / p.sum(axis=1).reshape(-1, 1)
q = q / q.sum(axis=1).reshape(-1, 1)
distances = (p[:, 0] - q[:, 0])

print(' '.join(ex['sentence1']))
print(' '.join(ex['sentence2']))

print("=========\n=======Farthest masks:=======")    
    
highlight_rankings = np.argsort(-distances)

for i in range(4):
    rank = highlight_rankings[i]
    m1_i, m2_i = mask_mapping[rank]
    
    masked1 = list(ex['sentence1'])
    for k in masks1[m1_i]:
        masked1[k] = '<m>'
    masked1 = ' '.join(masked1)
    
    masked2 = list(ex['sentence2'])
    for k in masks2[m2_i]:
        masked2[k] = '<m>'
    masked2 = ' '.join(masked2)
    
    print(masked1)
    print(masked2)
    print(np.round(distances[rank], 4))
